In [1]:
import json
import pickle as pkl
from sim.healthcare import *
import numpy as np
import numpy.random as rd

In [2]:
from sim.ebm.obj import load_obj_baseline
import pandas as pd
import numpy as np
from tqdm import tqdm
from sim.ebm.intervention import compose_intv

In [3]:
year0 = 2000

obj = load_obj_baseline(
    folder_input=f'pars',
    file_prior='data/prior.txt',
    file_targets='data/targets.csv',
    year0=year0,
    suffix='cas_cdx'
)

out_folder = 'out/post_dy_c0'

post = pd.read_csv(f'{out_folder}/Post.csv')

post = [dict(row) for i, row in post.iterrows()]

In [4]:
def get_pdx(sys, p_txi):
    res = sys.seek_care(1, 0)
    p_dx = np.array([res[sector].TruePos for sector in ['Public', 'Engaged', 'Private']])
    p_dx = (sys.Entry * p_dx * p_txi).sum()
    return p_dx

In [5]:
dd = list()

for pars in post[:30]:
    p = obj.serve(pars)
    p = obj.Cas.prepare_pars(p)

    intvs = {
        'Dx_TSwab': compose_intv(p, dx='TSwab'),
        'Dx_POC_Hi': compose_intv(p, dx='POC_Hi'),
        'Dx_POC': compose_intv(p, dx='POC'),
        
    }

    pdx = { 'Baseline': get_pdx(p['system'], p['p_txi']) }

    for k, v in intvs.items():
        pdx[k] = get_pdx(v.Dx.System, v.Dx.PrTxi)

    dd.append(pdx)
    
dd = pd.DataFrame(dd)

In [6]:
(dd.Baseline < dd.Dx_POC).mean()

1.0

In [7]:
(dd.Baseline < dd.Dx_POC_Hi).mean()

1.0

In [8]:
(dd.Baseline < dd.Dx_TSwab).mean()

1.0

In [9]:
p['system'].Public.seek_care(1, 0).TruePos, p['system'].Engaged.seek_care(1, 0).TruePos, p['system'].Private.seek_care(1, 0).TruePos

(0.9359179580662166, 0.7999214525599605, 0.7821)

In [10]:
intvs['Dx_POC_Hi'].Dx.System.Public.seek_care(1, 0).TruePos, intvs['Dx_POC_Hi'].Dx.System.Engaged.seek_care(1, 0).TruePos, intvs['Dx_POC_Hi'].Dx.System.Private.seek_care(1, 0).TruePos

(0.9359179580662166, 0.9259434998467501, 0.7821)

In [11]:
intvs['Dx_TSwab'].Dx.System.Public.seek_care(1, 0).TruePos, intvs['Dx_TSwab'].Dx.System.Engaged.seek_care(1, 0).TruePos, intvs['Dx_TSwab'].Dx.System.Private.seek_care(1, 0).TruePos

(0.9579055253929607, 0.80312952860778, 0.7821)

In [13]:
{
  "Presumptive": {
    "SSM": 13914911,
    "Xpert": 1370649 + 2749903
  },
  "Notification": {
    "Sector_Public": 1688427,
    "Sector_Private": 733694,

    "Test_SSM": 513050,
    "Test_Xpert": 548981,
    "Test_CXR": 835930,
    "Test_Other": 1037210,
    
    "Pr_XpertCases_Pri": (53067 + 68556) / (529196 + 525088)
  }
}

{'Presumptive': {'SSM': 13914911, 'Xpert': 4120552},
 'Notification': {'Sector_Public': 1688427,
  'Sector_Private': 733694,
  'Test_SSM': 513050,
  'Test_Xpert': 548981,
  'Test_CXR': 835930,
  'Test_Other': 1037210,
  'Pr_XpertCases_Pri': 0.11536075668415721}}

In [14]:
835930 + 1037210

1873140